# Ordinals

<https://docs.ordinals.com/>

<https://github.com/ordinals/ord/blob/master/bip.mediawiki>

核心是给所有挖出来的聪编号，从0开始。比如第一个block (height = 0)，也就是创世区块，挖出了50个btc，也就是 50_0000_0000 聪，那么这些聪的编号就是聪 0 到 49_9999_9999。

再有就是跟踪这些聪的转账历史，从而记录下每个聪的来龙去脉。

最终就是，对于一个utxo，我们需要知道，对于它所包含的币中的每个聪，其编号是什么。

这里的难点是跟踪聪的转账历史。btc本身并不区分同一个utxo中的所有聪，因此我们需要为其强行赋予一个输入到输出的映射关系。

直观上讲，比如我的一个utxo中包含了编号为1，3，5，7，9的聪，然后我支付3聪给A，1聪作为找零给我自己，还有1聪作为交易手续费给了矿工，那么A将得到1，3，5，我自己得到7，矿工得到9。这个非常符合直观，因此比较好理解。当然，确实有一些corner case需要考虑。

## 几个实验

### 1. 计算一个block挖出来的币的第一个聪的编号

In [34]:
# test data from f2pool.com
block_height = 574993
correct_ord = 1768741250000000

# 每4年减半，也就是没210000个区块减半，初始奖励为50BTC
def get_block_reward(block_height):
    answer = 50 * 10**8 / 2**(block_height // 210000)
    assert answer == int(answer)
    return int(answer)

def get_number_of_the_first_block_satoshi_brute_force(block_height):
    acc = 0
    for height in range(block_height):
        acc += get_block_reward(height)
    return acc

def get_number_of_the_first_block_satoshi(block_height):
    epoch = block_height // 210000 # 第几个4年周期
    acc = 0
    for e in range(epoch): # 每个epoch直接计算 
        acc += 210000 * get_block_reward(e * 210000)
    epoch_start = epoch * 210000
    epoch_offset = block_height - epoch_start
    epoch_reward = get_block_reward(epoch_start)
    acc += epoch_offset * epoch_reward
    return acc

calculated_ord = get_number_of_the_first_block_satoshi(block_height)
calculated_ord_brute_force = get_number_of_the_first_block_satoshi_brute_force(block_height)
assert calculated_ord == calculated_ord_brute_force
print('calculated_ord:', calculated_ord)
print('correct_ord:', correct_ord)
assert calculated_ord == correct_ord
print('OK')

calculated_ord: 1768741250000000
correct_ord: 1768741250000000
OK


### 2. 找到制定编号的聪目前在哪个UTXO中，归谁？

首先找到挖出它的block，然后根据转账历史一个tx一个tx的跟踪，直到找到当前的UTXO，从而找到其当前所有者。

因此一个聪经历的转账次数越多，找到他就越麻烦。比如我们来试着找一下看看。

In [86]:
import os, requests, json

# 测试地址
test_address = '39C7fxSzEACPjM78Z7xdPxhf7mKxJwvfMJ'
test_sat = 4529167619254
test_sat_block = 905

# from https://ordiscan.com/address/39C7fxSzEACPjM78Z7xdPxhf7mKxJwvfMJ/rare-sats we knows this sat belongs to this address

# 首先计算这个sat对应的block
def get_mined_block_number_from_sat(sat):
    epoch = 0
    epoch_start = epoch * 210000
    acc = 0
    epoch_full_reward = 210000 * get_block_reward(epoch_start)
    while acc + epoch_full_reward <= sat:
        acc += epoch_full_reward
        epoch += 1
        epoch_start += 210000
        epoch_full_reward = 210000 * get_block_reward(epoch_start)
    
    epoch_block_reward = get_block_reward(epoch_start)
    epoch_offset = (sat - acc) // epoch_block_reward
    block = epoch_start + epoch_offset
    return block

block = get_mined_block_number_from_sat(test_sat)
print('block:', block)
assert block == test_sat_block

# from https://ordiscan.com/sat/4529167619254 we knows this sat is mined in block 905

def get_cache_fn_content(fn):
    fn = f'ordinals-data/{fn}'
    # if file does not exists, return None
    if not os.path.exists(fn):
        return None
    with open(fn, 'r') as f:
        return f.read()

def put_cache_fn_content(fn, content):
    fn = f'ordinals-data/{fn}'
    with open(fn, 'w') as f:
        f.write(content)

web_request_cc = 0

def reset_web_request_cc():
    global web_request_cc
    web_request_cc = 0

def get_web_request_cc():
    global web_request_cc
    return web_request_cc

def get_web_content(url, fn=None, **options):
    global web_request_cc
    web_request_cc += 1

    if fn != None:
        fn_content = get_cache_fn_content(fn)
        if fn_content is not None:
            return fn_content
    print(f'fetching {url}')
    r = requests.get(url)
    content = None
    if 'allow_404' in options and options['allow_404'] == True:
        content = ''
    else:
        assert r.status_code == 200
        content = r.text
        assert content != None
    if fn != None:
        put_cache_fn_content(fn, content)
    return content

def get_block_hash_from_height(height):
    fn = f'block_hash_{height}.txt'
    url = f'https://mempool.space/api/block-height/{height}'
    hash = get_web_content(url, fn)
    assert len(hash) == 64
    return hash

def get_block_txs_from_height(height):
    hash = get_block_hash_from_height(height)
    url = f'https://mempool.space/api/block/{hash}/txs'
    fn = f'block_txs_{height}.txt'
    content = get_web_content(url, fn)

    obj = json.loads(content)
    assert len(obj) >= 1
    assert obj[0]['vin'][0]['is_coinbase'] == True
    return obj

def get_tx_from_txid(txid):
    assert len(txid) == 64
    url = f'https://mempool.space/api/tx/{txid}'
    fn = f'tx_{txid}.txt'
    content = get_web_content(url, fn)

    obj = json.loads(content)
    assert obj['txid'] == txid
    return obj

def fill_vin_of_tx(tx):
    assert len(tx['txid']) == 64

    for vin in tx['vin']:
        if vin['is_coinbase']:
            vin['prev_tx'] = None
            continue
        txid = vin['txid']
        vout = vin['vout']
        prev_tx = get_tx_from_txid(txid)
        vin['prev_tx'] = prev_tx
    
    return tx

def get_tx_spent_of_vout(txid, vout):
    assert len(txid) == 64
    assert vout >= 0

    fn = f'tx_spent_at_{txid}_{vout}.txt'
    url = f'https://mempool.space/api/tx/{txid}/outspend/{vout}'
    content = get_web_content(url, fn)
    
    obj = json.loads(content)

    if obj['spent'] == False:
        return None
    
    ret = (obj['txid'], obj['vin'])
    assert len(ret[0]) == 64
    assert ret[1] >= 0

    return ret

def get_tx_fee(txid):
    assert len(txid) == 64
    tx = get_tx_from_txid(txid)

    fee = tx['fee']
    assert fee >= 0
    return fee

    # fill_vin_of_tx(tx)

    # acc = 0
    # for vin in tx['vin']:
    #     assert 'prev_tx' in vin
    #     prev_tx = vin['prev_tx']
    #     value = prev_tx['vout'][vin['vout']]['value']
    #     acc += value
    
    # for vout in tx['vout']:
    #     acc -= vout['value']
    
    # assert acc >= 0

    # return acc

assert get_tx_fee('6db71e70bd0e5b9f2cc0678ff2bdb47215e2adc68dc72bdac973d93c76a54b7b') == 130_0000

def get_vout_for_offset(tx, offset):
    assert offset >= 0
    acc = 0
    idx = 0
    for vout in tx['vout']:
        if acc + vout['value'] > offset:
            return (idx, vout, offset - acc)
        acc += vout['value']
        idx += 1
    return None

def trace_sat(pos):
    txid = pos[0]
    assert len(txid) == 64
    offset = pos[1] # offset in all output sat's
    assert offset >= 0
    tx = get_tx_from_txid(txid)

    # find the output
    acc = 0
    tmp = get_vout_for_offset(tx, offset)

    correct_vout = None
    correct_n = 0
    correct_offset = 0
    if tmp != None:
        correct_n = tmp[0]
        correct_vout = tmp[1]
        correct_offset = tmp[2]
        assert correct_n >= 0
        assert correct_offset >= 0
    
    if tx['vin'][0]['is_coinbase'] == True and correct_vout == None:
        # destroyed
        print('destroyed')
        return None
    
    # not fee
    if correct_vout != None:
        spent_at = get_tx_spent_of_vout(txid, correct_n)
        if spent_at == None:
            # not yet spent
            return None

        spent_at_txid = spent_at[0]
        spent_at_vin_idx = spent_at[1]

        spent_at_tx = get_tx_from_txid(spent_at_txid)
        fill_vin_of_tx(spent_at_tx)

        acc = 0
        vin_found = False
        for vin in spent_at_tx['vin']:
            assert 'prev_tx' in vin
            prev_tx = vin['prev_tx']
            assert vin['txid'] == prev_tx['txid']

            if vin['txid'] == txid and vin['vout'] == correct_n:
                vin_found = True
                break

            acc += prev_tx['vout'][vin['vout']]['value']
        assert vin_found == True

        return (spent_at_txid, acc + correct_offset)
    
    #print('spent as fee')

    assert tx['status']['confirmed'] == True

    block_height = tx['status']['block_height']
    assert block_height >= 0
    block_hash = tx['status']['block_hash']
    assert len(block_hash) == 64

    block_reward = get_block_reward(block_height)
    txs = get_block_txs_from_height(block_height)

    coinbase_txid = txs[0]['txid']
    
    for tx in txs[1:]: # skip the coinbase tx
        if tx['txid'] == txid:
            break
        fee_of_this_tx = get_tx_fee(tx['txid'])
        block_reward += fee_of_this_tx
    
    return (coinbase_txid, block_reward + correct_offset)

reset_web_request_cc()

tx_list = get_block_txs_from_height(test_sat_block)
coinbase_tx = tx_list[0]

# current position of this sat is defined as (txid, output_offset)
cur_pos = (coinbase_tx['txid'], test_sat - get_number_of_the_first_block_satoshi(test_sat_block))

n_transfer = 0
while True:
    tx = get_tx_from_txid(cur_pos[0])
    assert tx['status']['confirmed'] == True
    block_height = tx['status']['block_height']
    print(f'cur_pos = {cur_pos}, cur_height = {block_height}, n_transfer = {n_transfer}')

    next_pos = trace_sat(cur_pos)
    if next_pos == None:
        break
    cur_pos = next_pos
    n_transfer += 1

print('final_pos', cur_pos)
print(f'total cost {get_web_request_cc()} web requests')



block: 905
cur_pos = ('4b4642c2f07835ca1299b7085c08d8702d66e659ff8dde9e6091661834860b19', 4167619254), cur_height = 905, n_transfer = 0
cur_pos = ('6db71e70bd0e5b9f2cc0678ff2bdb47215e2adc68dc72bdac973d93c76a54b7b', 699433619254), cur_height = 130796, n_transfer = 1
cur_pos = ('836cbc134a0ebfda370803a54806481da900da3053524d44b7f863d2e9ba9abb', 699415919254), cur_height = 188489, n_transfer = 2
cur_pos = ('46efa20155e5334f8c73262121307ff61e55fad4f3c9ac30dcb5a43b8a44dd70', 699415919254), cur_height = 192026, n_transfer = 3
cur_pos = ('c4de30ff3badbcb8f2f17dc0f2e65789f71e2eaa51c03b6f471efbf171273844', 699415919254), cur_height = 212094, n_transfer = 4
cur_pos = ('5bf2541a26a20248580b8c2258af5c087fb4e150fd3d6f01e3cf523e530b1963', 699415919254), cur_height = 217468, n_transfer = 5
cur_pos = ('820dba067b6e3d67ada05623425908809757ec1f1ceb1031782d6d5e2e2e0814', 699415919254), cur_height = 224348, n_transfer = 6
cur_pos = ('ad6b5cd64a150129909833d2d3ea79d221213090b5ce99395584f8d26a1afc98', 19941

### 3. 计算一个UTXO中包含的聪的所有编号？

一个当前的UTXO，表示的是一个到现在为止还没有被花费掉的 TX OUTPUT。

一个历史的UTXO，表示的是历史上该 TX OUTPUT没有被花费掉，后来某个时刻被花费掉了。

一个UTXO包含了多少聪，就有多少个聪编号。比如一个UTXO包含了一个btc，也就是 $ 10^8 $ 聪，那就是 $10^8$ 个编号。一一列举出这些编号，往往是复杂的。

不过，对于某些UTXO，这事儿还是简单，比如一个空块儿的coinbase的output。这些output的聪编号是连续的，所以知道了第一个聪的编号，就能直接知道所有聪的编号。这里说到空块儿，是因为如果不是空块儿，那么coinbase的output中就会包含所有交易的手续费中包含的聪，那这些聪编号就复杂了。